In [3]:
import json
import os
import glob
import re
import pandas as pd
from os import makedirs
from os.path import join, exists
from pathlib import Path
import numpy as np

In [19]:
states = (('US', -1), ('US-AL', 1), ('US-AK', 2), ('US-AZ', 4), ('US-AR', 5), ('US-CA', 6), ('US-CO', 8), \
          ('US-CT', 9), ('US-DE', 10), ('US-DC', 11), ('US-FL', 12), ('US-GA', 13), \
         ('US-HI', 15), ('US-ID', 16), ('US-IL', 17), ('US-IN', 18), ('US-IA', 19), ('US-KS', 20), \
          ('US-KY', 21), ('US-LA', 22), ('US-ME', 23), ('US-MD', 24), ('US-MA', 25), ('US-MI', 26), \
         ('US-MN', 27), ('US-MS', 28), ('US-MO', 29), ('US-MT', 30), ('US-NE', 31), \
          ('US-NV', 32), ('US-NH', 33), ('US-NJ', 34), ('US-NM', 35), ('US-NY', 36), \
          ('US-NC', 37), ('US-ND', 38), ('US-OH', 39), ('US-OK', 40), ('US-OR', 41), ('US-PA', 42), \
          ('US-RI', 44), ('US-SC', 45), ('US-SD', 46), ('US-TN', 47), ('US-TX', 48), ('US-UT', 49), \
          ('US-VT', 50), ('US-VA', 51), ('US-WA', 53), ('US-WV', 54), ('US-WI', 55), ('US-WY', 56))


In [14]:
xx = [('dd', 1)]

In [17]:
xx[0][0]

'dd'

In [20]:
def get_candidates(yr):
    if yr == '2012':
        return 'obama', 'romney', 'BO', 'MR'
    elif yr == '2016':
        return 'hillary', 'trump', 'HC', 'DT'
    else:
        return 'biden', 'trump', 'JB', 'DT'

In [21]:
def get_winner_by_topic_and_year(tp, yr):
    if tp == 'economy':
        if yr == '2012':
            return 'romney'
        if yr == '2016':
            return 'trump'
        if yr == '2020':
            return 'biden'
    elif tp == 'immigration':
        if yr == '2012':
            return 'obama'
        if yr == '2016':
            return 'hillary' 
        if yr == '2020':
            return 'biden'
    elif tp == 'health':
        if yr == '2012':
            return 'romney'
        if yr == '2016':
            return 'hillary' 
        if yr == '2020':
            return 'biden'
    elif tp == 'environment':
        if yr == '2012':
            return 'obama'
        if yr == '2016':
            return 'hillary' 
        if yr == '2020':
            return 'biden'

In [22]:
def deep_red_blue_states(state_fip):
    deep_red = [0, 3, 12, 26, 34, 41, 50]
    deep_blue = [4, 32, 47]
    
    if state_fip in deep_red:
        return 1
    elif state_fip in deep_blue:
        return 0
    else:
        return -1

In [23]:
def get_state_fips(states, st):
    for s in states:
        if s[0] == st:
            return s[1]

In [24]:
def agg_all(states, yr):
    topics = ['economy', 'environment', 'health', 'immigration']
    
    result = []
    combined_result = []
    
    econ_result = []
    env_result = []
    health_result = []
    imm_result = []
    
    for tp in topics:
        
        cand1, cand2, cs1, cs2 = get_candidates(yr)

        target_path_c1 = join('cand-state-agg', tp, yr, cand1)
        target_path_c2 = join('cand-state-agg', tp, yr, cand2)

        df1 = pd.read_csv(join(target_path_c1, 'trends.csv'))
        df2 = pd.read_csv(join(target_path_c2, 'trends.csv'))
        
        win_cand = get_winner_by_topic_and_year(tp, yr)
        
        for index, row in df1.iterrows():
            cand1_score = row['score']
            cand2_score = float(df2.iloc[index].score)
            
            deep_state_score = deep_red_blue_states(index - 1)
            
            ultimate_winner = win_cand
            
            st_fip = get_state_fips(states, row['state'])
            
            if deep_state_score == -1:
                ultimate_winner = cs1 if cand1 == win_cand else cs2
            else:
                ultimate_winner = cs1 if deep_state_score == 0 else cs2
            
            if tp == 'economy':
                econ_result.append({'state_fips': st_fip, 'state': row['state'], 'year': yr, \
                                    '_' + tp: cs1 if cand1_score > cand2_score else cs2, \
                                    tp: ultimate_winner, \
                                   'score': cand1_score if cand1_score > cand2_score else cand2_score})
            elif tp == 'environment':
                env_result.append({'state_fips': st_fip, 'year': yr, \
                                    '_' + tp: cs1 if cand1_score > cand2_score else cs2, \
                                   tp: ultimate_winner, \
                                   'score': cand1_score if cand1_score > cand2_score else cand2_score})
            elif tp == 'health':
                health_result.append({'state_fips': st_fip, 'year': yr, \
                                    '_' + tp: cs1 if cand1_score > cand2_score else cs2, \
                                    tp: ultimate_winner, \
                                   'score': cand1_score if cand1_score > cand2_score else cand2_score})
            elif tp == 'immigration':
                imm_result.append({'state_fips': st_fip, 'year': yr, \
                                    '_' + tp: cs1 if cand1_score > cand2_score else cs2, \
                                   tp: ultimate_winner, \
                                   'score': cand1_score if cand1_score > cand2_score else cand2_score})
            
        
    
    for x in range(52):
        st_fip = env_result[x]['state_fips']
        yr = env_result[x]['year']
        
        combined_result.append({ 
            'year': yr, 'state_fips': econ_result[x]['state_fips'], 'state': econ_result[x]['state'],
            'economy': econ_result[x]['economy'],
            'environment': env_result[x]['environment'],
            'health': health_result[x]['health'],
            'immigration': imm_result[x]['immigration']
        })
    
    print(pd.DataFrame(combined_result))
    
    return pd.DataFrame(combined_result)

In [25]:
result_20 = agg_all(states, '2020')

    year  state_fips  state economy environment health immigration
0   2020          -1     US      JB          JB     JB          JB
1   2020           1  US-AL      DT          DT     DT          DT
2   2020           2  US-AK      JB          JB     JB          JB
3   2020           4  US-AZ      JB          JB     JB          JB
4   2020           5  US-AR      DT          DT     DT          DT
5   2020           6  US-CA      JB          JB     JB          JB
6   2020           8  US-CO      JB          JB     JB          JB
7   2020           9  US-CT      JB          JB     JB          JB
8   2020          10  US-DE      JB          JB     JB          JB
9   2020          11  US-DC      JB          JB     JB          JB
10  2020          12  US-FL      JB          JB     JB          JB
11  2020          13  US-GA      JB          JB     JB          JB
12  2020          15  US-HI      JB          JB     JB          JB
13  2020          16  US-ID      DT          DT     DT        

In [26]:
result_16 = agg_all(states, '2016')

    year  state_fips  state economy environment health immigration
0   2016          -1     US      DT          HC     HC          HC
1   2016           1  US-AL      DT          DT     DT          DT
2   2016           2  US-AK      DT          HC     HC          HC
3   2016           4  US-AZ      DT          HC     HC          HC
4   2016           5  US-AR      DT          DT     DT          DT
5   2016           6  US-CA      HC          HC     HC          HC
6   2016           8  US-CO      DT          HC     HC          HC
7   2016           9  US-CT      DT          HC     HC          HC
8   2016          10  US-DE      DT          HC     HC          HC
9   2016          11  US-DC      DT          HC     HC          HC
10  2016          12  US-FL      DT          HC     HC          HC
11  2016          13  US-GA      DT          HC     HC          HC
12  2016          15  US-HI      DT          HC     HC          HC
13  2016          16  US-ID      DT          DT     DT        

In [27]:
result_12 = agg_all(states, '2012')

    year  state_fips  state economy environment health immigration
0   2012          -1     US      MR          BO     MR          BO
1   2012           1  US-AL      MR          MR     MR          MR
2   2012           2  US-AK      MR          BO     MR          BO
3   2012           4  US-AZ      MR          BO     MR          BO
4   2012           5  US-AR      MR          MR     MR          MR
5   2012           6  US-CA      BO          BO     BO          BO
6   2012           8  US-CO      MR          BO     MR          BO
7   2012           9  US-CT      MR          BO     MR          BO
8   2012          10  US-DE      MR          BO     MR          BO
9   2012          11  US-DC      MR          BO     MR          BO
10  2012          12  US-FL      MR          BO     MR          BO
11  2012          13  US-GA      MR          BO     MR          BO
12  2012          15  US-HI      MR          BO     MR          BO
13  2012          16  US-ID      MR          MR     MR        

In [28]:
result_combined = pd.concat([result_20, result_16, result_12])

In [29]:
result_combined

,year,state_fips,state,economy,environment,health,immigration
0,2020,-1,US,JB,JB,JB,JB
1,2020,1,US-AL,DT,DT,DT,DT
2,2020,2,US-AK,JB,JB,JB,JB
3,2020,4,US-AZ,JB,JB,JB,JB
4,2020,5,US-AR,DT,DT,DT,DT
...,...,...,...,...,...,...,...
47,2012,51,US-VA,MR,BO,MR,BO
48,2012,53,US-WA,BO,BO,BO,BO
49,2012,54,US-WV,MR,BO,MR,BO
50,2012,55,US-WI,MR,BO,MR,BO


In [30]:
result_combined.to_csv('cand-state-agg/top_candidates.csv')